# Nairobi Multiple Deprivation Indicators - Preprocessing

This code was developed by Eqi Luo, for his MSc Research. In this code, we would prepare and standardize the deprivation incicators for the PCA.  

In [1]:
# Firstly, import the necessary packages for this analysis

import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage import exposure
import os
from osgeo import gdal



In [2]:
# define dataset directory
BASE_PATH = r"C:\Users\EQiLu\MSc Thesis\100m Input Mid-term"
# set the working path
os.chdir(BASE_PATH)

## Part1: loading, checking, structring the input dataset

In [3]:
# Define a function to read the first band of geotiff file as a numpy array
# Here we only read the first band because each indicator layer has only one single band! 

def gtiff_to_array(file_path):
    "read the first band and return a 2-D array of width * height"
    data = gdal.Open(file_path)
    band = data.GetRasterBand(1).ReadAsArray()
    return band

In [4]:
# Read all the geotiff files and convert them into np arrays
# Each tiff layer is one indicator representing multiple deprivation

skilled_birth = gtiff_to_array('nai_skilledbirth_atten_2014.tif')
poverty = gtiff_to_array('nai_poverty_2008.tif')
women_literacy = gtiff_to_array('nai_women_literacy_2014.tif')
men_literacy = gtiff_to_array('nai_men_literacy_2014.tif')
vaccination = gtiff_to_array('nai_dt3_vaccination_2014.tif')
access_itn = gtiff_to_array('nai_access_itn_2014.tif')
stunted_child = gtiff_to_array('nai_stuned_children_2014.tif')
family_planning = gtiff_to_array('nai_unmet_family_plan_2014.tif')
impr_housing = gtiff_to_array('nai_imprvd_housing_2015.tif')
water_source = gtiff_to_array('nai_water_source_2014.tif')
open_defecation = gtiff_to_array('nai_open_defecation_2014.tif')
pit_latrines = gtiff_to_array('nai_density_pit_latrines.tif')
armed_conflicts = gtiff_to_array('nai_density_crime_2019.tif')
pm_25 = gtiff_to_array('nai_pm25_2016.tif')
waterways = gtiff_to_array('nai_water_density_2020.tif')
dump_sites = gtiff_to_array('nai_density_dumpsites_2017.tif')
pop_density = gtiff_to_array('nai_pop_density_2020.tif')
bld_density = gtiff_to_array('nai_bld_density_2020.tif')
ndvi = gtiff_to_array('nai_ndvi_2019.tif')
night_light = gtiff_to_array('nai_nightlight_2016.tif')
bus_stations = gtiff_to_array('nai_den_bus_stop_2019.tif')
dis_roads = gtiff_to_array('nai_OSM_dis_major_roads_2020.tif')
poor_roads = gtiff_to_array('nai_poor_road_density_2020.tif')
dis_education = gtiff_to_array('nai_dis_education_2020.tif')
dis_health = gtiff_to_array('nai_dis_health_2020.tif')
dis_financial = gtiff_to_array('nai_dis_financial_2020.tif')
built_up = gtiff_to_array('nai_grid3_builtup_2020.tif')
slum = gtiff_to_array('slum_boundary_nairobi.tif')

In [5]:
# Check the sahpe of each geotiff file. 
# The standarized size of each raster layer should be 434*253(height * width)

print(skilled_birth.shape)
print(poverty.shape)
print(women_literacy.shape)
print(men_literacy.shape)
print(vaccination.shape)
print(access_itn.shape)
print(stunted_child.shape)
print(family_planning.shape)
print(impr_housing.shape)
print(water_source.shape)
print(open_defecation.shape)
print(pit_latrines.shape)
print(armed_conflicts.shape)
print(pm_25.shape)
print(waterways.shape)
print(dump_sites.shape)
print(pop_density.shape)
print(bld_density.shape)
print(ndvi.shape)
print(night_light.shape)
print(bus_stations.shape)
print(dis_roads.shape)
print(poor_roads.shape)
print(dis_education.shape)
print(dis_health.shape)
print(dis_financial.shape)


print(built_up.shape)
print(slum.shape)



(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)
(253, 434)


In [6]:
# Stack the indicator layers into a 3-d array, where the shape is NUM_INDICATORS * HEIGHT * WIDTH

# Note that we could also use np.concatenate function to compile the layers into a stack, but with some modification 

dataset = np.stack((skilled_birth, poverty, women_literacy, men_literacy, vaccination, access_itn, stunted_child, family_planning, 
                    impr_housing, water_source, open_defecation, pit_latrines, armed_conflicts, pm_25, waterways, dump_sites,
                    pop_density, bld_density, ndvi, night_light, bus_stations, dis_roads, poor_roads, dis_education, dis_health,
                    dis_financial, built_up, slum),axis = 0)

In [7]:
# Now we check the shape of the dataset
print(dataset.shape)
print(dataset)

(28, 253, 434)
[[[-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  ...
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]]

 [[-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  ...
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]]

 [[-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  ...
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. -99999. -99999. ... -99999. -99999. -99999.]
  [-99999. 

In [8]:
## Create an array which stores the indices (x,y coordinate information) of each pixel and concatenate it into the dataset

index = np.indices((253,434))
dataset = np.concatenate((dataset,index),axis=0)
print(dataset.shape)
print(dataset)

# After this step, the new array shape should be 30+2(with two more layers storing the coordinates) * height * width 

(30, 253, 434)
[[[-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  ...
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]]

 [[-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  ...
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.9999e+04
   -9.9999e+04]
  [-9.9999e+04 -9.9999e+04 -9.9999e+04 ... -9.9999e+04 -9.

In [9]:
# Mask allthe non-data value. Note that in this case, the nondata value of all indicators is -99999

dataset = np.ma.masked_equal(dataset,-99999)
print(dataset) 

[[[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 ...

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 [[0.0 0.0 0.0 ... 0.0 0.0 0.0]
  [1.0 1.0 1.0 ... 1.0 1.0 1.0]
  [2.0 2.0 2.0 ... 2.0 2.0 2.0]
  ...
  [250.0 250.0 250.0 ... 250.0 250.0 250.0]
  [251.0 251.0 251.0 ... 251.0 251.0 251.0]
  [252.0 252.0 252.0 ... 252.0 252.0 252.0]]

 [[0.0 1.0 2.0 ... 431.0 432.0 433.0]
  [0.0 1.0 2.0 ... 431.0 432.0 433.0]
  [0.0 1.0 2.0 ... 431.0 4

In [10]:
# We need to transform the 3-d array into 2-d array, since later it will be the input of clustering analysis. 
# Now the shape should be (number of indicators+2, width*height)

dataset = dataset.reshape(30,-1)

print(dataset)
print(dataset.shape)

[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-- -- -- ... -- -- --]
 [0.0 0.0 0.0 ... 252.0 252.0 252.0]
 [0.0 1.0 2.0 ... 431.0 432.0 433.0]]
(30, 109802)


In [11]:
# Transpose the array - as now the column should be the indicators, whereas the rows represent the individual pixel

training_data = np.transpose(dataset)
print(training_data.shape)
print(training_data)

(109802, 30)
[[-- -- -- ... -- 0.0 0.0]
 [-- -- -- ... -- 0.0 1.0]
 [-- -- -- ... -- 0.0 2.0]
 ...
 [-- -- -- ... -- 252.0 431.0]
 [-- -- -- ... -- 252.0 432.0]
 [-- -- -- ... -- 252.0 433.0]]


In [12]:
# Convert the data from numpy array into dataframe
# Add the lable to each corresponding column(indicator) 

labels = ['skilled_birth', 'poverty', 'women_literacy', 'men_literacy', 'vaccination', 'access_itn', 'stunted_child', 'family_planning',
          'impr_housing', 'water_source', 'open_defecation', 'pit_latrines', 'armed_conflicts', 'pm_25', 'waterways', 'dump_sites',
          'pop_density', 'bld_density', 'ndvi', 'night_light', 'bus_stations', 'dis_roads', 'poor_roads', 'dis_education', 'dis_health',
          'dis_financial', 'built_up', 'slum', 'rows', 'columns']

df = pd.DataFrame(training_data, columns = labels) 
print(df)

        skilled_birth  poverty  women_literacy  men_literacy  vaccination  \
0                 NaN      NaN             NaN           NaN          NaN   
1                 NaN      NaN             NaN           NaN          NaN   
2                 NaN      NaN             NaN           NaN          NaN   
3                 NaN      NaN             NaN           NaN          NaN   
4                 NaN      NaN             NaN           NaN          NaN   
...               ...      ...             ...           ...          ...   
109797            NaN      NaN             NaN           NaN          NaN   
109798            NaN      NaN             NaN           NaN          NaN   
109799            NaN      NaN             NaN           NaN          NaN   
109800            NaN      NaN             NaN           NaN          NaN   
109801            NaN      NaN             NaN           NaN          NaN   

        access_itn  stunted_child  family_planning  impr_housing  \
0      

In [13]:
# Drop the rows which contians any 'Nondata'

df = df.dropna()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62888 entries, 275 to 109498
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   skilled_birth    62888 non-null  float64
 1   poverty          62888 non-null  float64
 2   women_literacy   62888 non-null  float64
 3   men_literacy     62888 non-null  float64
 4   vaccination      62888 non-null  float64
 5   access_itn       62888 non-null  float64
 6   stunted_child    62888 non-null  float64
 7   family_planning  62888 non-null  float64
 8   impr_housing     62888 non-null  float64
 9   water_source     62888 non-null  float64
 10  open_defecation  62888 non-null  float64
 11  pit_latrines     62888 non-null  float64
 12  armed_conflicts  62888 non-null  float64
 13  pm_25            62888 non-null  float64
 14  waterways        62888 non-null  float64
 15  dump_sites       62888 non-null  float64
 16  pop_density      62888 non-null  float64
 17  bld_densi

In [14]:
df.describe()

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,...,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial,built_up,slum,rows,columns
count,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,...,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000,62888.000000
mean,0.733944,0.087087,0.945702,0.930647,0.758267,0.390458,0.144080,0.069822,0.617168,0.980663,...,5.639918,1024.676028,1.088303,1339.037163,2228.350538,1625.973784,1.272898,1.970837,123.807865,209.419889
std,0.006172,0.047273,0.017933,0.010149,0.034434,0.100261,0.018497,0.007009,0.080535,0.012195,...,10.341506,913.695894,1.460026,1230.105571,1994.888937,1299.752568,0.445452,0.168264,51.244319,111.698579
min,0.702342,0.002084,0.907093,0.891019,0.663898,0.210798,0.110470,0.054260,0.347661,0.893894,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.730828,0.050088,0.929869,0.923430,0.733857,0.297587,0.130703,0.064729,0.562751,0.974618,...,0.000000,360.555115,0.144366,500.000000,806.225769,632.455505,1.000000,2.000000,85.000000,111.000000
50%,0.735716,0.088813,0.946521,0.932282,0.751328,0.404965,0.143776,0.070187,0.624130,0.984406,...,0.000000,800.000000,0.572383,921.954468,1697.056274,1280.624878,1.000000,2.000000,122.000000,216.000000
75%,0.738669,0.109609,0.963856,0.938222,0.785709,0.464139,0.153868,0.074143,0.677595,0.988734,...,7.417899,1442.220459,1.386329,1772.004517,3000.000000,2262.741699,2.000000,2.000000,164.000000,298.000000
max,0.741470,0.259697,0.973960,0.949055,0.846470,0.596174,0.213720,0.090319,0.783054,0.995236,...,98.722023,5830.951660,12.452886,7006.425781,11412.711914,8420.213867,2.000000,2.000000,252.000000,433.000000


In [15]:
print(df)

        skilled_birth   poverty  women_literacy  men_literacy  vaccination  \
275          0.732475  0.139231        0.939562      0.915470     0.794153   
276          0.732762  0.138914        0.939609      0.915438     0.794153   
277          0.732935  0.138598        0.939609      0.915438     0.794150   
278          0.733172  0.138281        0.939609      0.915500     0.794150   
279          0.733441  0.137965        0.939632      0.915500     0.794150   
...               ...       ...             ...           ...          ...   
109494       0.737453  0.103516        0.973103      0.926475     0.762950   
109495       0.738017  0.105053        0.973103      0.926475     0.764222   
109496       0.738769  0.106591        0.972991      0.926967     0.764222   
109497       0.739287  0.108128        0.972991      0.926967     0.765465   
109498       0.739779  0.109666        0.972991      0.927393     0.765465   

        access_itn  stunted_child  family_planning  impr_housin

## Split the dataset into built-up and non-builtup

Built up dataset

In [16]:
df_builtup = df[df['built_up'] == 1]
print(df_builtup.shape)

(45726, 30)


In [17]:
# Standardize and save the indicators before running PCA 

df_builtup.iloc[:,0:26] = StandardScaler().fit_transform(df_builtup.iloc[:,0:26])

C:\Users\EQiLu\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-17-22b72d57eda5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_builtup.iloc[:,0:26] = StandardScaler().fit_transform(df_builtup.iloc[:,0:26])


In [18]:
df_builtup.describe()

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,...,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial,built_up,slum,rows,columns
count,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,...,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,4.572600e+04,45726.0,45726.000000,45726.000000,45726.000000
mean,5.456863e-15,1.449333e-15,-4.793902e-13,-1.189399e-12,1.286598e-15,7.710599e-14,-2.857028e-14,-4.112400e-14,6.812553e-16,3.408850e-15,...,4.438022e-14,3.938881e-15,-4.346031e-16,4.156604e-15,-1.950308e-15,3.133958e-15,1.0,1.960066,121.850129,195.647115
std,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,...,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,0.0,0.195805,48.931926,107.971445
min,-5.067489e+00,-1.715603e+00,-2.178689e+00,-3.414483e+00,-2.655319e+00,-1.708250e+00,-2.070906e+00,-2.206159e+00,-3.332640e+00,-6.567502e+00,...,-6.363746e-01,-1.216246e+00,-6.967286e-01,-1.069795e+00,-1.057688e+00,-1.205358e+00,1.0,1.000000,1.000000,0.000000
25%,-4.741504e-01,-8.443559e-01,-9.344946e-01,-7.977365e-01,-7.214668e-01,-8.794731e-01,-6.647242e-01,-7.164890e-01,-6.806262e-01,-6.034111e-01,...,-6.363746e-01,-7.870521e-01,-5.968144e-01,-6.663477e-01,-6.889071e-01,-7.637079e-01,1.0,2.000000,86.000000,104.000000
50%,3.248833e-01,4.895283e-02,2.831324e-01,2.029637e-01,-2.115470e-01,5.046951e-02,-1.413990e-02,5.077904e-02,1.387328e-01,2.935500e-01,...,-5.562085e-01,-2.147933e-01,-3.364583e-01,-3.016540e-01,-2.939564e-01,-2.846311e-01,1.0,2.000000,120.000000,193.000000
75%,7.370598e-01,5.570529e-01,9.501543e-01,7.631349e-01,8.023417e-01,7.924609e-01,5.316022e-01,6.351650e-01,7.554268e-01,7.222240e-01,...,3.225120e-01,5.763495e-01,1.571132e-01,3.020748e-01,3.727083e-01,4.607091e-01,1.0,2.000000,155.000000,278.000000
max,1.159465e+00,3.288444e+00,1.483544e+00,1.877760e+00,2.446225e+00,2.099215e+00,3.179610e+00,2.936538e+00,1.972611e+00,1.310318e+00,...,7.993462e+00,5.650860e+00,7.940236e+00,5.778423e+00,4.517560e+00,4.338660e+00,1.0,2.000000,252.000000,433.000000


In [19]:
df_builtup.to_csv('0501_builtup_indicators.csv')

In [20]:
# Visualize the correlation matrix of each indicator 

correlation1 = df_builtup.iloc[:,0:26].corr()
correlation1.style.background_gradient(cmap='coolwarm').set_precision(4)

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,open_defecation,pit_latrines,armed_conflicts,pm_25,waterways,dump_sites,pop_density,bld_density,ndvi,night_light,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial
skilled_birth,1.0000,-0.6508,0.0000,0.4263,0.0450,-0.0520,-0.0439,-0.0749,0.2729,0.5779,-0.5441,0.1526,0.1051,0.3668,0.2305,0.2618,0.3147,0.2507,-0.3133,0.4935,0.3568,-0.3174,0.1379,-0.4128,-0.6990,-0.5567
poverty,-0.6508,1.0000,-0.1520,-0.7149,-0.0038,0.2425,-0.0679,0.1560,-0.3150,-0.5695,0.4153,-0.2273,-0.1441,-0.2334,-0.2774,-0.2877,-0.3600,-0.2171,0.3192,-0.7073,-0.4342,0.3127,-0.1507,0.6420,0.7616,0.5953
women_literacy,0.0000,-0.1520,1.0000,-0.0851,-0.7626,-0.7091,0.2584,-0.1227,-0.1839,0.1453,0.2128,-0.0368,-0.0209,-0.4357,-0.1941,-0.1952,-0.2675,-0.1867,0.3814,-0.2591,-0.0879,-0.0140,-0.0682,-0.1773,-0.1150,0.0544
men_literacy,0.4263,-0.7149,-0.0851,1.0000,0.0819,-0.0798,0.0648,-0.3143,0.3059,0.2546,-0.3593,0.1759,0.0764,0.2598,0.2047,0.2466,0.2634,0.1391,-0.3759,0.6669,0.3310,-0.3265,0.0978,-0.3881,-0.6010,-0.5845
vaccination,0.0450,-0.0038,-0.7626,0.0819,1.0000,0.5700,-0.1412,0.2055,0.0841,0.0353,-0.1649,0.0710,0.0595,0.3865,0.2626,0.2107,0.3053,0.1772,-0.3606,0.3442,0.1992,-0.0353,0.0176,0.1167,0.1160,-0.0783
access_itn,-0.0520,0.2425,-0.7091,-0.0798,0.5700,1.0000,-0.4475,0.0489,0.0413,-0.0884,-0.2570,-0.0071,-0.0084,0.5945,0.0914,0.0883,0.1072,0.1102,-0.3503,0.1087,-0.0160,0.1055,0.0871,0.2623,0.1284,-0.1138
stunted_child,-0.0439,-0.0679,0.2584,0.0648,-0.1412,-0.4475,1.0000,0.3978,-0.1700,-0.0094,0.4909,0.1697,0.0084,-0.2519,-0.0437,0.0677,0.0412,-0.0696,0.0867,0.0807,0.0535,-0.1473,0.0481,-0.0558,0.0722,-0.0030
family_planning,-0.0749,0.1560,-0.1227,-0.3143,0.2055,0.0489,0.3978,1.0000,0.1404,0.0541,0.3779,0.1451,0.1246,0.0289,0.0647,0.0862,0.0506,-0.0520,0.0840,0.0330,0.0030,0.0551,0.0936,0.0522,0.1853,-0.0097
impr_housing,0.2729,-0.3150,-0.1839,0.3059,0.0841,0.0413,-0.1700,0.1404,1.0000,0.1400,-0.2379,-0.0819,0.1098,0.2985,0.2367,0.0555,0.0845,0.0051,-0.0659,0.3163,0.1087,-0.1762,-0.0215,-0.3285,-0.4607,-0.4977
water_source,0.5779,-0.5695,0.1453,0.2546,0.0353,-0.0884,-0.0094,0.0541,0.1400,1.0000,-0.4378,0.1729,0.0923,0.0961,0.2123,0.2465,0.2592,0.2154,-0.2095,0.3918,0.2480,-0.2206,0.1351,-0.4515,-0.5994,-0.4363


Non built-up dataset

In [21]:
df_non_builtup = df[df['built_up'] == 2]

In [22]:
print(df_non_builtup.shape)

(17162, 30)


In [23]:
df_non_builtup.iloc[:,0:27] = StandardScaler().fit_transform(df_non_builtup.iloc[:,0:27])

C:\Users\EQiLu\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-23-37df415a1b06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_builtup.iloc[:,0:27] = StandardScaler().fit_transform(df_non_builtup.iloc[:,0:27])


In [24]:
df_non_builtup.to_csv('0501_non_builtup_indicators.csv')

In [25]:
df_non_builtup.describe()

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,...,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial,built_up,slum,rows,columns
count,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,...,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,1.716200e+04,17162.0,17162.000000,17162.000000,17162.000000
mean,2.225621e-16,-2.135143e-16,2.901672e-13,-2.408274e-13,-2.660731e-16,-3.527756e-14,2.020280e-14,-4.103504e-14,-6.278805e-16,2.240541e-15,...,3.133540e-16,7.332052e-16,1.892428e-15,-5.665877e-16,2.116811e-16,1.111387e-17,0.0,1.999534,129.024007,246.115721
std,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,...,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,0.0,0.021586,56.620912,113.166749
min,-4.267417e+00,-2.306817e+00,-1.969181e+00,-3.339261e+00,-3.029937e+00,-2.080520e+00,-1.340131e+00,-2.243776e+00,-3.439215e+00,-5.014653e+00,...,-3.085297e-01,-1.232923e+00,-8.833975e-01,-1.412823e+00,-1.378424e+00,-1.551899e+00,0.0,1.000000,0.000000,0.000000
25%,-5.424146e-01,-6.056461e-01,-7.420749e-01,-5.411376e-01,-6.666068e-01,-8.304516e-01,-8.585777e-01,-7.569047e-01,-6.831994e-01,-3.351817e-01,...,-3.085297e-01,-7.503392e-01,-7.852978e-01,-7.868204e-01,-6.981392e-01,-7.476635e-01,0.0,2.000000,83.000000,151.000000
50%,2.239732e-01,-1.046601e-01,-2.471506e-01,1.950436e-01,-1.721361e-01,2.026926e-01,-1.128579e-01,4.849249e-02,-1.444189e-03,2.188391e-01,...,-3.085297e-01,-2.677553e-01,-3.605847e-01,-3.264306e-01,-2.446157e-01,-2.271702e-01,0.0,2.000000,132.000000,282.000000
75%,7.554537e-01,4.429155e-01,1.113199e+00,6.436959e-01,7.251640e-01,5.735127e-01,5.355824e-01,5.651955e-01,6.719690e-01,7.187936e-01,...,-3.085297e-01,4.712601e-01,4.565428e-01,6.931573e-01,3.720528e-01,5.352888e-01,0.0,2.000000,179.000000,332.000000
max,1.474515e+00,3.194609e+00,1.884710e+00,1.709940e+00,2.953001e+00,1.925209e+00,3.297747e+00,2.886071e+00,2.259517e+00,1.260185e+00,...,1.361115e+01,3.592916e+00,4.573736e+00,3.344507e+00,3.797508e+00,4.254902e+00,0.0,2.000000,252.000000,433.000000


Whole Nairobi 

In [26]:
df1 = df

In [27]:
df1.iloc[:,0:26] = StandardScaler().fit_transform(df1.iloc[:,0:26])

In [28]:
df1.describe()

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,...,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial,built_up,slum,rows,columns
count,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,...,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,6.288800e+04,62888.000000,62888.000000,62888.000000,62888.000000
mean,-8.331226e-15,8.659468e-16,1.609159e-12,5.121240e-13,1.923891e-15,3.440806e-14,-1.132994e-13,-1.369240e-13,-1.484245e-15,1.465315e-16,...,2.151058e-14,-1.659763e-15,2.096630e-15,-6.009693e-15,-2.322154e-15,-1.169650e-14,1.272898,1.970837,123.807865,209.419889
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,...,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,0.445452,0.168264,51.244319,111.698579
min,-5.120573e+00,-1.798154e+00,-2.153008e+00,-3.904610e+00,-2.740627e+00,-1.791943e+00,-1.817070e+00,-2.220281e+00,-3.346499e+00,-7.115267e+00,...,-5.453715e-01,-1.121472e+00,-7.454058e-01,-1.088563e+00,-1.117039e+00,-1.250997e+00,1.000000,1.000000,0.000000,0.000000
25%,-5.049035e-01,-7.826765e-01,-8.829280e-01,-7.110804e-01,-7.089180e-01,-9.263003e-01,-7.231584e-01,-7.266153e-01,-6.757071e-01,-4.957095e-01,...,-5.453715e-01,-7.268569e-01,-6.465262e-01,-6.820909e-01,-7.128899e-01,-7.643964e-01,1.000000,2.000000,85.000000,111.000000
50%,2.870783e-01,3.650574e-02,4.566171e-02,1.610857e-01,-2.015317e-01,1.447001e-01,-1.638907e-02,5.201586e-02,8.644706e-02,3.069411e-01,...,-5.453715e-01,-2.459000e-01,-3.533665e-01,-3.390652e-01,-2.663299e-01,-2.657057e-01,1.000000,2.000000,122.000000,216.000000
75%,7.654805e-01,4.764374e-01,1.012306e+00,7.463783e-01,7.969729e-01,7.348973e-01,5.291846e-01,6.164740e-01,7.503298e-01,6.618575e-01,...,1.719281e-01,4.569877e-01,2.041253e-01,3.519786e-01,3.868163e-01,4.899186e-01,2.000000,2.000000,164.000000,298.000000
max,1.219347e+00,3.651382e+00,1.575750e+00,1.813804e+00,2.561574e+00,2.051824e+00,3.765038e+00,2.924372e+00,2.059826e+00,1.194985e+00,...,9.000898e+00,5.260299e+00,7.783884e+00,4.607274e+00,4.603983e+00,5.227375e+00,2.000000,2.000000,252.000000,433.000000


In [29]:
df1.to_csv('0510_whole_nairobi_indicators.csv')

In [31]:
# Visualize the correlation matrix of each indicator 

correlation = df_builtup.iloc[:,0:26].corr()
correlation.style.background_gradient(cmap='coolwarm').set_precision(2)

,skilled_birth,poverty,women_literacy,men_literacy,vaccination,access_itn,stunted_child,family_planning,impr_housing,water_source,open_defecation,pit_latrines,armed_conflicts,pm_25,waterways,dump_sites,pop_density,bld_density,ndvi,night_light,bus_stations,dis_roads,poor_roads,dis_education,dis_health,dis_financial
skilled_birth,1.00,-0.65,0.00,0.43,0.05,-0.05,-0.04,-0.07,0.27,0.58,-0.54,0.15,0.11,0.37,0.23,0.26,0.31,0.25,-0.31,0.49,0.36,-0.32,0.14,-0.41,-0.70,-0.56
poverty,-0.65,1.00,-0.15,-0.71,-0.00,0.24,-0.07,0.16,-0.32,-0.57,0.42,-0.23,-0.14,-0.23,-0.28,-0.29,-0.36,-0.22,0.32,-0.71,-0.43,0.31,-0.15,0.64,0.76,0.60
women_literacy,0.00,-0.15,1.00,-0.09,-0.76,-0.71,0.26,-0.12,-0.18,0.15,0.21,-0.04,-0.02,-0.44,-0.19,-0.20,-0.27,-0.19,0.38,-0.26,-0.09,-0.01,-0.07,-0.18,-0.12,0.05
men_literacy,0.43,-0.71,-0.09,1.00,0.08,-0.08,0.06,-0.31,0.31,0.25,-0.36,0.18,0.08,0.26,0.20,0.25,0.26,0.14,-0.38,0.67,0.33,-0.33,0.10,-0.39,-0.60,-0.58
vaccination,0.05,-0.00,-0.76,0.08,1.00,0.57,-0.14,0.21,0.08,0.04,-0.16,0.07,0.06,0.39,0.26,0.21,0.31,0.18,-0.36,0.34,0.20,-0.04,0.02,0.12,0.12,-0.08
access_itn,-0.05,0.24,-0.71,-0.08,0.57,1.00,-0.45,0.05,0.04,-0.09,-0.26,-0.01,-0.01,0.59,0.09,0.09,0.11,0.11,-0.35,0.11,-0.02,0.11,0.09,0.26,0.13,-0.11
stunted_child,-0.04,-0.07,0.26,0.06,-0.14,-0.45,1.00,0.40,-0.17,-0.01,0.49,0.17,0.01,-0.25,-0.04,0.07,0.04,-0.07,0.09,0.08,0.05,-0.15,0.05,-0.06,0.07,-0.00
family_planning,-0.07,0.16,-0.12,-0.31,0.21,0.05,0.40,1.00,0.14,0.05,0.38,0.15,0.12,0.03,0.06,0.09,0.05,-0.05,0.08,0.03,0.00,0.06,0.09,0.05,0.19,-0.01
impr_housing,0.27,-0.32,-0.18,0.31,0.08,0.04,-0.17,0.14,1.00,0.14,-0.24,-0.08,0.11,0.30,0.24,0.06,0.08,0.01,-0.07,0.32,0.11,-0.18,-0.02,-0.33,-0.46,-0.50
water_source,0.58,-0.57,0.15,0.25,0.04,-0.09,-0.01,0.05,0.14,1.00,-0.44,0.17,0.09,0.10,0.21,0.25,0.26,0.22,-0.21,0.39,0.25,-0.22,0.14,-0.45,-0.60,-0.44
